# Counting reads in features with `htseq-count`

Given a file with aligned sequencing reads and a list of **genomic features**, a common task is to **count how many reads map to each feature**.

A **feature** is here an interval (i.e., a range of positions) on a chromosome or a union of such intervals.

In the case of *RNA-Seq*, the features are typically **genes**, where each gene is considered here as the union of all its exons. One may also consider each **exon** as a feature, e.g., in order to check for alternative splicing. For comparative *ChIP-Seq*, the features might be **binding regions** from a pre-determined list.

Special care must be taken to decide how to deal with **reads** that align to or **overlap with more than one feature**. The `htseq-count` **script** allows to choose between three modes. Of course, if none of these fits your needs, you can write your own script with *HTSeq*. See the chapter [A tour through HTSeq](https://htseq.readthedocs.io/en/release_0.11.1/tour.html#tour) for a step-by-step guide on how to do so. See also the FAQ at the end, if the following explanation seems too technical.

The three overlap resolution modes of `htseq-count` work as follows. For each position i in the **read**, a set S(i) is defined as the **set of all features overlapping position i**. Then, consider the set S, which is (with *i* running through all position within the read or a read pair)

* the **union** of all the sets S(i) for mode `union`. This mode is recommended for most use cases.
* the **intersection** of all the sets S(i) for mode `intersection-strict`.
* the **intersection** of all **non-empty** sets S(i) for mode `intersection-nonempty`.

If S contains **precisely one feature**, the read (or read pair) is counted for this feature. If S is **empty**, the read (or read pair) is counted as `no_feature`. If S contains **more than one feature**, `htseq-count` behaves differently based on the `--nonunique` **option**:

* `--nonunique none` (default): the **read** (or read pair) is counted as `ambiguous` and **not counted for any features**. Also, if the read (or read pair) **aligns to more than one** location in the reference, it is scored as `alignment_not_unique`.

* --nonunique all: the **read** (or read pair) is counted as `ambiguous` and is also counted in **all features** to which it was assigned. Also, if the read (or read pair) **aligns to more than one location** in the reference, it is scored as `alignment_not_unique` and also **separately for each location**.

Notice that when using `--nonunique` all the sum of all counts will not be equal to the number of reads (or read pairs), because those with **multiple alignments** or **overlaps** get scored multiple times.

The following figure illustrates the effect of these three modes and the `--nonunique` **option**:

![img_count](https://htseq.readthedocs.io/en/release_0.11.1/_images/count_modes.png)

## Usage

After you have installed HTSeq (see [Prequisites and installation](https://htseq.readthedocs.io/en/release_0.11.1/install.html#install)), you can run `htseq-count` from the command line:

```
htseq-count [options] <alignment_files> <gff_file>
```

If the file `htseq-count` is not in your path, you can, alternatively, call the script with:

````
python -m HTSeq.scripts.count [options] <alignment_files> <gff_file>
```

The `<alignment_files>` are one or more files containing the aligned reads in `SAM` or `BAM` format (see **option** `-f <format>`, `--format=<format>.` If nedded, `SAMtools` contains Perl scripts to convert most alignment formats to SAM/BAM. Make sure to use a **splicing-aware** aligner such as `STAR`. HTSeq-count makes full use of the information in the **CIGAR field**.

To read from **standard input**, use `-` as `<alignment_files>`.

If you have paired-end data, pay attention to the `-r` **option** described below.

The `<gff_file>` contains the features in the [GFF format](http://gmod.org/wiki/GFF2).

The script outputs **a table with counts** for **each feature**, followed by the **special counters**, which count reads that **were not counted for any feature for various reasons**. The names of the special counters all start with a double underscore, to facilitate filtering. (Note: The double unscore was absent up to `version 0.5.4`). 

The **special counters** are:

* `__no_feature`: **reads** (or read pairs) which could **not** be **assigned to any feature** (set S as described above was **empty**).

* `__ambiguous`: **reads** (or read pairs) which could have been **assigned to more than one feature** and hence were not counted for any of these, unless the `--nonunique` all option was used (set S had **more than one element**).

* `__too_low_aQual`: **reads** (or read pairs) which were **skipped** due to the `-a` **option**, see below.

* `__not_aligned`: **reads** (or read pairs) in the `SAM` file **without alignment**.

* `__alignment_not_unique`: **reads** (or read pairs) with **more than one reported alignment**. These reads are recognized from the `NH` (number of hits) optional `SAM` field tag. If the aligner does not set this field, multiply aligned reads will be counted multiple times, unless they get filtered out by due to the `-a` **option**. Note that if the `--nonunique all` **option** was used, these reads (or read pairs) are still assigned to features.

**Important**: The default for **strandedness** is `yes`. If your RNA-Seq data has not been made with a **strand-specific protocol**, this causes half of the reads to be lost. Hence, make sure to set the **option** `--stranded=no` unless you have strand-specific data!

## Options

`-f <format>`, `--format=<format>`
* Format of the input data. Possible values are `sam` (for **text** `SAM` **files**) and `bam` (for **binary** `BAM` **files**). (**default**: `sam`)

`-r <order>`, `--order=<order>`
* For paired-end data, the alignment have to be sorted either by read name or by alignment position. If your data is not sorted, use the `samtools sort` function of `samtools` to sort it. Use this option, with `name` or `pos` for `<order>` to indicate how the input data has been sorted. (**default**: `name`)

* If `name` is indicated, `htseq-count` expects all the alignments for the reads of a given read pair to appear in **adjacent records** in the input data. For `pos`, this is not expected; rather, read alignments whose mate alignment have not yet been seen are kept in a buffer in memory until the mate is found. While, strictly speaking, the latter will also work with **unsorted data**, sorting ensures that most alignment mates appear close to each other in the data and hence the buffer is much less likely to overflow.

`--max-reads-in-buffer=<number>`
* When `<alignment_file>` is paired end sorted by position, allow only so many reads to stay in memory until the mates are found (raising this number will use more memory). **Has no effect for single end** or **paired end sorted by name**. (**default**: `30000000`)

`-s <yes/no/reverse>`, `--stranded=<yes/no/reverse>`
* whether the data is from a **strand-specific assay** (**default**: `yes`)

* For `stranded=no`, a read is considered overlapping with a feature regardless of whether it is **mapped to the same** or the **opposite strand** as the feature. For `stranded=yes` and single-end reads, the **read has to be mapped to the same strand as the feature**. For paired-end reads, the first read has to be on the same strand and the second read on the opposite strand. For `stranded=reverse`, these rules are reversed.

`-a <minaqual>`, `--a=<minaqual>`
* skip all reads with alignment quality lower than the given minimum value (**default**: `10` — Note: the default used to be `0` until `version 0.5.4.`)

`-t <feature type>`, `--type=<feature type>`
* **feature type** (3rd column in `GFF` **file**) to be used, all features of other type are ignored (**default**, suitable for RNA-Seq analysis using an Ensembl `GTF` **file**: `exon`)

`-i <id attribute>`, `--idattr=<id attribute>`
* `GFF` **attribute** to be used as **feature ID**. Several `GFF` **lines** with the **same feature ID** will be considered as **parts of the same feature**. The **feature ID** is used to identity the **counts** in the **output table**. The **default**, suitable for RNA-Seq analysis using an Ensembl `GTF` **file**, is `gene_id`.

`--additional-attr=<id attributes>`
* **Additional feature attributes**, which will be printed as an additional column after the primary attribute column but before the counts column(s). The **default** is `none`, a suitable value to get gene names using an Ensembl `GTF` **file** is `gene_name`. To use more than one additional attribute, repeat the option in the command line more than once, with a single attribute each time, e.g. `--additional-attr=gene_name --additional_attr=exon_number`.

`-m <mode>`, `--mode=<mode>`
* **Mode** to handle **reads overlapping more than one feature**. Possible values for `<mode>` are `union`, `intersection-strict` and `intersection-nonempty` (**default**: `union`)

`--nonunique=<nonunique mode>`
* **Mode** to handle **reads that align** to or are assigned **to more than one feature in the overlap** `<mode>` of choice (see `-m` option). `<nonunique mode>` are `none` and `all` (**default**: `none`)

`--secondary-alignments=<mode>`
* Mode to handle **secondary alignments** (`SAM flag 0x100`). `<mode>` can be score and ignore (**default**: `score`)

`--supplementary-alignments=<mode>`
* Mode to handle **supplementary/chimeric alignments** (`SAM flag 0x800`). `<mode>` can be score and ignore (**default**: `score`)

`-o <samout>`, `--samout=<samout>`
* **write out** all `SAM` **alignment records** into an output `SAM` file called `<samout>`, annotating each line with its assignment to a **feature** or a **special counter** (as an **optional field** with tag `‘XF’`)

`-q`, `--quiet`
* suppress **progress report** and **warnings**

`-h`, `--help`
* Show a usage summary and exit

## FAQ

<font color='red'> To be continued ... </font>